In [2]:
# diff of llm and chatAi
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# text-davinchi-003 is stopped
llm = OpenAI(model_name = "gpt-3.5-turbo-instruct")
# model_name = "gpt-3.5-turbo"
chat = ChatOpenAI()

a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

a,b

('\n\nThere are currently eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. There may be more planets yet to be discovered in our solar system and beyond.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")

# chatAi options
chat = ChatOpenAI(
    temperature=0.1,
    streaming= True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# hard code message
#messages = [
#   SystemMessage(content= "You are a geography expert. And you only reply in korean"),
#   AIMessage(content= "안녕하세요 내 친구여"),
#   HumanMessage(content= "what is the distance between Korea and Italy. Also what is your name?")
#]

#chat.predict_messages(messages)

messages = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}"),
        ("ai", "안녕하세요 내 친구여 저는 {name}이라 합니다"),
        ("human", "what is the distance between {country_a} and {country_b}. Also what is your name?")
    ]
)

#prompt = template.format(country_a = "Japan", country_b = "America")
#chat.predict(prompt)

prompt = messages.format_messages(
    language = "Korean",
    name = "Jung pal",
    country_a = "Korea",
    country_b = "Thailand"
)

chat.predict_messages(prompt)

한국과 태국 사이의 거리는 약 3,200km입니다. 제 이름은 정팔입니다. 어떻게 도와드릴까요?

AIMessageChunk(content='한국과 태국 사이의 거리는 약 3,200km입니다. 제 이름은 정팔입니다. 어떻게 도와드릴까요?')

In [23]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()

#p.parse("Hello, how, are, you")

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. And do not reply with anything else."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items = 10,
    question = "What are the planets?"
)

result = chat.predict_messages(prompt)

p.parse(result.content)

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [26]:
# chain all upper operation
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 10,
    "question": "What are the digimon"
})

['Agumon',
 'Gabumon',
 'Patamon',
 'Gatomon',
 'Biyomon',
 'Tentomon',
 'Palmon',
 'Gomamon',
 'Veemon',
 'Renamon']

In [4]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients"),
    ("human", "I want to cook {cuisine} food")
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food then just say you don't know how to replace it. And answer in korean"),
    ("human", "{recipe}")
])

veg_chef_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chef_chain

final_chain.invoke({
    "cuisine": "indian",
})

Great choice! Indian cuisine is full of bold flavors and aromatic spices. Let's start with a classic recipe for Chicken Tikka Masala. 

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 tbsp garam masala
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1/2 tsp turmeric
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)
- Cooked rice or naan bread (to serve)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, 1 tsp garam masala, 1/2 tsp cumin, 1/2 tsp coriander, paprika, turmeric, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well. Marinate for at least 1 hour, preferably overnight.



AIMessageChunk(content='인도 요리 중에서 대표적인 치킨 틱카 마살라 레시피를 채식 버전으로 준비해보겠습니다.\n\n대체재료:\n- 대체육 (예를 들어 대체육이나 대체닭)\n- 두유 요거트\n- 레몬즙\n- 식물성 오일\n- 양파\n- 다진 마늘\n- 생강\n- 토마토 통조림\n- 가람 마살라\n- 고춧가루\n- 생활용 소금\n- 후추\n- 신선한 코리앤더\n\n준비과정:\n1. 대체육을 사용하여 기존의 닭고기를 대체합니다. 대체육을 적당한 크기로 자르고, 두유 요거트, 레몬즙, 식물성 오일, 다진 마늘, 생강, 가람 마살라, 고춧가루, 소금, 후추를 넣어 잘 섞어주세요. 적어도 1시간 이상, 가급적 하룻밤 동안 재워주세요.\n\n2. 오븐을 200도로 예열합니다. 대체육을 꼬치에 꿴 후 베이킹 시트에 올려 20-25분간 굽거나 익을 때까지 굽니다.\n\n3. 대형 프라이팬에 식물성 오일을 데워 양파를 넣고 5분 정도 볶아 부드럽게 익힙니다. 나머지 다진 마늘과 생강을 넣고 1분 정도 더 볶아주세요.\n\n4. 토마토 통조림, 가람 마살라, 고춧가루, 소금을 넣고 가볍게 끓여 10-15분 정도 끓입니다.\n\n5. 굽은 대체육을 프라이팬에 넣고 5-10분 정도 더 끓여 각종 양념이 잘 어우러지도록 합니다. 필요에 따라 소금과 후추로 맛을 조절해주세요.\n\n6. 쌀밥이나 나안 빵과 함께 대체육 틱카 마살라를 내놓고 신선한 코리앤더로 장식하여 즐겨보세요. 맛있는 홈메이드 인도 요리를 즐기세요!')